# Import Required Libraries
Import the necessary libraries

In [14]:
# %pip install pandas networkx community scikit-learn pyvis ipython-sql

In [15]:
import networkx as nx
import pandas as pd
import community.community_louvain as community_louvain
from sklearn.preprocessing import MinMaxScaler
from pyvis.network import Network

# Load Data from Database
Load data from the SQLite database into a pandas DataFrame using a SQL query. The query selects pairs of artists who have collaborated on tracks, along with the count of their collaborations. The data is filtered based on the popularity of the artists and tracks.

In [16]:
# Define the filters to apply to the data
artist_popularity = 60 # The minimum popularity of an artist
track_popularity = 30 # The minimum popularity of a track

In [17]:
# Load the ipython-sql extension
%load_ext sql

# Connect to the SQLite database
%sql sqlite:///../db/spotify.sqlite

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [18]:
%%sql
SELECT a1.name AS artist_1,
    a2.name AS artist_2,
    COUNT(*) AS collaboration_count
FROM TrackArtist ta1
JOIN TrackArtist ta2 ON ta1.track_id = ta2.track_id
AND ta1.artist_id < ta2.artist_id
JOIN Artist a1 ON ta1.artist_id = a1.id
JOIN Artist a2 ON ta2.artist_id = a2.id
JOIN Track ON ta1.track_id = Track.id
WHERE a1.name IS NOT NULL
AND a2.name IS NOT NULL
AND Track.popularity > :track_popularity
AND a1.popularity > :artist_popularity
AND a2.popularity > :artist_popularity
GROUP BY a1.name, a2.name
ORDER BY collaboration_count DESC

 * sqlite:///../db/spotify.sqlite
Done.


artist_1,artist_2,collaboration_count
Metro Boomin,Future,39
Rampa,&ME,38
Adam Port,&ME,37
Rampa,Adam Port,37
Keinemusik,Adam Port,33
Dimitri Vegas,Dimitri Vegas & Like Mike,32
Rampa,Keinemusik,32
Keinemusik,&ME,30
Ellie Goulding,Calvin Harris,26
Macklemore,Ryan Lewis,25


In [19]:
# Load output from the cell above
result = _ # type: ignore

# Convert the result to a DataFrame
data_frame = result.DataFrame()

# Display the first few rows of the DataFrame
data_frame.head()

,artist_1,artist_2,collaboration_count
0,Metro Boomin,Future,39
1,Rampa,&ME,38
2,Adam Port,&ME,37
3,Rampa,Adam Port,37
4,Keinemusik,Adam Port,33


# Show tracks
List the first couple of tracks of the artist pair with the highest collaboration count.

In [20]:
# Define the specific artist pair
artist_1, artist_2 = data_frame.iloc[0]["artist_1"], data_frame.iloc[0]["artist_2"]

In [21]:
%%sql
SELECT DISTINCT
  Track.name AS track_name,
  Track.id AS track_id
FROM Track
JOIN TrackArtist ON Track.id = TrackArtist.track_id
JOIN Artist ON TrackArtist.artist_id = Artist.id
WHERE Artist.name IN (:artist_1, :artist_2)
  AND Track.popularity > :track_popularity
  AND Artist.popularity > :artist_popularity

 * sqlite:///../db/spotify.sqlite
Done.


track_name,track_id
All I Know,0NWqNXBJTpXbkI5rPWNy3p
My Darlin' (feat. Future),7hJ1fEr1nf2e4qsE4bfmcc
Cold (feat. Future),2NlTOhsAamXOaZciOXbITb
Thrusting (feat. Swae Lee & Future),4ZrutH1rzpA4v3Bg9nma55
Turn On The Lights again.. (feat. Future & Fred again..) - Anyma Remix,4ptnQ0kQnN1U1Ig8TSslj6
Turn On The Lights again.. (feat. Future & Fred again..) - Austin Millz Remix,1F8hDR8MLN24m2bCWkAWuz
"Fine Whine (feat. Joe Fox, Future & M.I.A.)",2WxzLbJfPA2vJGxQiKd7Jq
On Time (with John Legend),0YFqKxV9uNu6LUeYkLOKRS
Superhero (Heroes & Villains) [with Future & Chris Brown],0vjeOZ3Ft5jvAi9SBFJm1j
Too Many Nights (feat. Don Toliver & with Future),2Hh3ETdQKrmSI3QS0hme7g


In [22]:
# Load the output from the cell above
result = _ # type: ignore

# Execute the query and load the data into a DataFrame
top_tracks_df = result.DataFrame()

# Display the top tracks
top_tracks_df.head()

,track_name,track_id
0,All I Know,0NWqNXBJTpXbkI5rPWNy3p
1,My Darlin' (feat. Future),7hJ1fEr1nf2e4qsE4bfmcc
2,Cold (feat. Future),2NlTOhsAamXOaZciOXbITb
3,Thrusting (feat. Swae Lee & Future),4ZrutH1rzpA4v3Bg9nma55
4,Turn On The Lights again.. (feat. Future & Fre...,4ptnQ0kQnN1U1Ig8TSslj6


# Create Collaboration Graph
Create a NetworkX graph from the DataFrame, filtering edges based on a minimum number of collaborations.

In [23]:
# Create NetworkX graph from the DataFrame, filtering edges based on a minimum number of collaborations
def create_graph(data_frame: pd.DataFrame, min_collaborations: int = 3) -> nx.Graph:
    G = nx.Graph()
    
    # Use itertuples() for better performance
    for row in data_frame.itertuples(index=False):
        G.add_edge(row.artist_1, row.artist_2, weight=row.collaboration_count)

    # Filter low-weight edges
    filtered_edges = [(u, v) for u, v, d in G.edges(data=True) if d["weight"] > min_collaborations]
    return G.edge_subgraph(filtered_edges).copy()

# Create graph with a minimum collaboration threshold
G_filtered = create_graph(data_frame, min_collaborations=4)
print(f"Nodes: {G_filtered.number_of_nodes()}, Edges: {G_filtered.number_of_edges()}")

Nodes: 244, Edges: 273


# Calculate Centrality Measures
Calculate degree centrality and betweenness centrality for the nodes in the graph.

In [24]:
# Sets centrality values to 1 for max to 0 for min
def normalize_centrality(centrality_dict):
    values = list(centrality_dict.values())
    scaler = MinMaxScaler()
    normalized_values = scaler.fit_transform([[v] for v in values]).flatten()
    
    return dict(zip(centrality_dict.keys(), normalized_values))

# Degree and betweenness centrality
degree_centrality = normalize_centrality(nx.degree_centrality(G_filtered))
betweenness_centrality = normalize_centrality(nx.betweenness_centrality(G_filtered))

# Display top 5 artists
print("\n🎵 Top 5 Artists by Degree Centrality:")
for artist, score in sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"{artist}: {score:.3f}")

print("\n🔗 Top 5 Artists by Betweenness Centrality:")
for artist, score in sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"{artist}: {score:.3f}")


🎵 Top 5 Artists by Degree Centrality:
David Guetta: 1.000
The Weeknd: 1.000
Justin Bieber: 1.000
Metro Boomin: 0.833
Travis Scott: 0.750

🔗 Top 5 Artists by Betweenness Centrality:
David Guetta: 1.000
The Weeknd: 0.641
Nicki Minaj: 0.621
Justin Bieber: 0.571
Kygo: 0.463


# Detect Communities
Use the Louvain method to detect communities within the graph and print the number of detected communities.

In [25]:
# Use the Louvain method to detect communities within the graph
partition = community_louvain.best_partition(G_filtered)

# Print the number of detected communities
num_communities = len(set(partition.values()))
print(f"Detected {num_communities} artist communities.")

Detected 53 artist communities.


# Visualize Collaboration Network
Each dot represents an artist. Each line represents a relation, thicker lines represent more collaborations between the artists.

In [30]:
# Create a Pyvis network visualization
def visualize_graph_pyvis(G: nx.Graph):
    net = Network(notebook=True, width="100%", height="700px", bgcolor="#222222", font_color="white")
    
    for node in G.nodes():
        net.add_node(node, title=node, color="blue")
    
    for edge in G.edges(data=True):
        net.add_edge(edge[0], edge[1], value=edge[2].get('weight', 1))

    net.toggle_physics(True)  # Enable physics for better layout
    
    net.set_options("""
        var options = {
          "nodes": { "shape": "dot", "size": 10 },
          "edges": { "color": "gray", "width": 1 },
          "physics": {
            "barnesHut": { "gravitationalConstant": -30000, "centralGravity": 0.3 }
          }
        }
    """)
    
    net.show("artist_network.html")

visualize_graph_pyvis(G_filtered)


artist_network.html
